# ML HW1

In [ ]:
import numpy as np
import pandas as pd

data_x_df = pd.read_csv('X.csv')
data_t_df = pd.read_csv('T.csv')
data_df = pd.concat([data_x_df, data_t_df], axis=1)
data_df

In [ ]:
dataset = np.array(data_df)
np.random.shuffle(dataset)

train_ratio = 0.9
train_size = int(len(dataset) * train_ratio)

train_feature = dataset[:train_size, :-1]
test_feature = dataset[train_size:, :-1]
train_target = dataset[:train_size, -1]
test_target = dataset[train_size:, -1]

## 1. Feature Selection

In [ ]:
def optimal_weight(phi, target):
    return np.linalg.pinv(phi) @ target

### M = 1

$$ y(\textbf{x}, \textbf{w}) = \textit{w}_{0} + \sum_{\textit{i} = 1}^{D} \textit{w}_{i}\textit{x}_{i}  = \textbf{w}^{T} \phi(x) $$
where
$$ \textbf{w} = \begin{pmatrix} \textit{w}_{0} & \textit{w}_{1} & ... & \textit{w}_{D} \end{pmatrix}^{T} $$
$$ \phi(x) = \begin{pmatrix} 1 & \textit{x}_{1} & ... & \textit{x}_{D}  \end{pmatrix}^{T} $$

In [ ]:
def phi_m1(features):
    """Computes phi matrix with m = 1.
    
    Args:
        features: np.ndarray with shape (n_samples, n_features)
        
    Returns:
        phi: np.ndarray with shape (n_samples, 1 + n_features)
    """
    m_0 = np.ones((features.shape[0], 1))  # 1
    m_1 = features  # x_1, x_2, ..., x_D
    return np.concatenate((m_0, m_1), axis=-1)

In [ ]:
train_phi_m1 = phi_m1(train_feature)
test_phi_m1 = phi_m1(test_feature)

w_m1 = optimal_weight(train_phi_m1, train_target)


### M = 2

$$ y(\textbf{x}, \textbf{w}) = \textit{w}_{0} + \sum_{\textit{i} = 1}^{D} \textit{w}_{i}\textit{x}_{i} + \sum_{\textit{i} = 1}^{D} \sum_{\textit{j} = 1}^{D} \textit{w}_{ij}\textit{x}_{i}\textit{x}_{j} = \textbf{w}^{T} \phi(x) $$
where
$$ \textbf{w} = \begin{pmatrix} \textit{w}_{0} & \textit{w}_{1} & ... & \textit{w}_{D} & \textit{w}_{11} & \textit{w}_{12} & ... & \textit{w}_{DD} \end{pmatrix}^{T} $$
$$ \phi(x) = \begin{pmatrix} 1 & \textit{x}_{1} & ... & \textit{x}_{D} & \textit{x}_{1}\textit{x}_{1} & \textit{x}_{1}\textit{x}_{2} & ... & \textit{x}_{D}\textit{x}_{D} \end{pmatrix}^{T} $$

In [ ]:
def phi_m2(features):
    """Computes phi matrix with m = 2.
    
    Args:
        features: np.ndarray with shape (n_samples, n_features)
        
    Returns:
        phi: np.ndarray with shape (n_samples, 1 + n_features + n_features ** 2)
    """
    m_0 = np.ones((features.shape[0], 1))  # 1
    m_1 = features  # x_1, x_2, ..., x_D

    # x_1^2, x_1x_2, ..., x_1x_D, x_2^2, x_2x_3, ..., x_2x_D, ..., x_D^2
    m_2 = np.expand_dims(features, axis=-1) @ np.expand_dims(features, axis=-2)
    m_2 = m_2.reshape(m_2.shape[0], -1)
    return np.concatenate((m_0, m_1, m_2), axis=-1)

In [ ]:
train_phi_m2 = phi_m2(train_feature)
test_phi_m2 = phi_m2(test_feature)

w_m2 = optimal_weight(train_phi_m2, train_target)

### Root Mean Square Error

In [ ]:
def root_mean_square_error(y, t):
    return np.sqrt(np.mean((y - t)**2))


train_rms_m1 = root_mean_square_error(train_phi_m1 @ w_m1, train_target)
test_rms_m1 = root_mean_square_error(test_phi_m1 @ w_m1, test_target)

train_rms_m2 = root_mean_square_error(train_phi_m2 @ w_m2, train_target)
test_rms_m2 = root_mean_square_error(test_phi_m2 @ w_m2, test_target)

In [ ]:
print(f'train rms m1: {train_rms_m1}')
print(f'test rms m1: {test_rms_m1}')
print(f'train rms m2: {train_rms_m2}')
print(f'test rms m2: {test_rms_m2}')

### Weight Analysis of M = 1

I used the weight of the model to analyze the importance of each feature. The weight of the model is the coefficient of each feature. The larger the weight is, the more important the feature is.

> The weight with index 0 is not considered as a feature because it is the bias of the model.

In [ ]:
print(f'w_m1: {w_m1}')
abs_importance = np.abs(w_m1[1:])
importance_rank = np.argsort(abs_importance)[::-1]
print(f'Rank of features\' contribution: {importance_rank}')
print(f'The most important feature: {data_df.columns[importance_rank[0]]}')

## 2. Maximum Likelihood Approach

In [ ]:
def get_mean_and_std(features):
    """Computes the mean and standard deviation of each feature.
    
    Args:
        features: np.ndarray with shape (n_samples, n_features)
        
    Returns:
        mean: np.ndarray with shape (1, n_features)
        std: np.ndarray with shape (1, n_features)
    """
    mean = np.mean(features, axis=0, keepdims=True)
    std = np.std(features, axis=0, keepdims=True)
    return mean, std


def phi_gaussian(features, mean, std):
    """Computes the Gaussian phi.
    
    Args:
        features: np.ndarray with shape (n_samples, n_features)
        mean: np.ndarray with shape (1, n_features)
        std: np.ndarray with shape (1, n_features)
        
    Returns:
        phi: np.ndarray with shape (n_samples, 1 + n_features)
    """
    gaussians = np.exp(-((features - mean) / std)**2 / 2)
    return np.concatenate((np.ones((features.shape[0], 1)), gaussians), axis=-1)

In [ ]:
train_mean, train_std = get_mean_and_std(train_feature)
train_phi_gaussian = phi_gaussian(train_feature, train_mean, train_std)
test_phi_gaussian = phi_gaussian(test_feature, train_mean, train_std)

w_gaussian = optimal_weight(train_phi_gaussian, train_target)

In [ ]:
train_rms_gaussian = root_mean_square_error(train_phi_gaussian @ w_gaussian, train_target)
test_rms_gaussian = root_mean_square_error(test_phi_gaussian @ w_gaussian, test_target)

print(f'train rms gaussian: {train_rms_gaussian}')
print(f'test rms gaussian: {test_rms_gaussian}')

## 3. Maximum A Posteriori Approach